In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train = pd.read_csv('../yemoonsaBigdata/datasets/Part2/census_X_train.csv')
y_train = pd.read_csv('../yemoonsaBigdata/datasets/Part2/census_y_train.csv')

X_test = pd.read_csv('../yemoonsaBigdata/datasets/Part2/census_X_test.csv')

<br>

### **데이터 전처리**

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education_num   30162 non-null  int64 
 3   marital_status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   capital_gain    30162 non-null  int64 
 9   capital_loss    30162 non-null  int64 
 10  hours_per_week  30162 non-null  int64 
 11  native_country  30162 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.8+ MB


<br>

출력된 정보를 보면 12개의 column이 있다.

전체 30,162행이 있고, 결측치는 없는 것으로 보인다.

age, education_num, captial_gain, captial_loss, hours_per_week 변수의 데이터 타입은 `Int64`, 그 외 변수의 데이터 타입은 `object`이다.

변수 설명과 변수의 데이터 타입이 일치하므로 별도의 데이터 타입 변경은 필요 없을 것으로 보인다.

다만, 문자열로 된 변수는 모델 학습 전 인코딩 작업이 필요하다.

<br>


In [4]:
X_train.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,10.121312,1092.007858,88.372489,40.931238
std,13.134665,2.549995,7406.346497,404.298370,11.979984
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,47.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


<br>

또한 `describe` 함수를 통해 평균, 표준편차, 사분위수 등 기본적인 요약 통계량 값을 확인한다.

captial_gain, captial_loss 두 변수는 데이터가 오른쪽으로 크게 치우쳐져 있는 것을 확인할 수 있다.

두 변수의 75 백분위수 ~ 100 백분위수 범위를 5 백분위수 단위로 쪼개어서 자세히 살펴보자.

<br>

In [8]:
X_train['capital_gain'].quantile([q/20 for q in range(15, 21)])

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64

In [9]:
X_train['capital_loss'].quantile([q/20 for q in range(15, 21)])

0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64

<br>

capital_gain 변수는 전체 10% 이내, capital_loss 변수는 전체 5% 이내만 0보다 큰 값을 가진다.

이 경우 변수 그대로 사용하거나, 데이터 변환을 하거나, 범주형 변수로 변환하여 사용할 수 있다.

<br>

이번에는 범주형 변수로 변환하여 새로운 파생변수를 만들고, 초도 모델링 이후 변수 중요도를 통해 기존 변수와 파생변수 중 어떤 변수가 더 중요하게 쓰이는지 확인해보려고 한다.

numpy 패키지의 `where` 함수를 이용해서 파생변수 capital_gain_yn, capital_loss_yn 을 만들도록 하자.

<br>

In [25]:
X_train['capital_gain_yn'] = np.where(X_train['capital_gain'] > 0, 1, 0)
X_train['capital_loss_yn'] = np.where(X_train['capital_loss'] > 0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain'] > 0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss'] > 0, 1, 0)

<br>

종속변수 값 별로 독립변수 값의 차이가 있는지 탐색하기 위해 변수 종류별로 나눠주자.

<br>

In [26]:
columns_remove = []
columns_num = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'capital_loss']
columns_cat = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'capital_gain_yn', 'capital_loss_yn']
columns_y = ['target']

In [27]:
X_train = X_train.drop(columns_remove, axis=1)
X_test = X_test.drop(columns_remove, axis=1)

In [28]:
train_df = pd.concat([X_train, y_train], axis=1)

for col in columns_num:
    print("-" * 80)
    print(col)
    print(train_df.groupby(columns_y)[col].describe(), end='\n\n')

--------------------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       22654.0  39.348592  11.950774  1.0  38.0  40.0  

<br>

age, education_num, hours_per_week 변수 모두 <font color='tomato'> 종속변수 값이 1일 때의 평균이 0일 때의 평균보다 더 크다. </font>

capital_gain, capital_loss는 종속변수 값에 따른 <font color='tomato'> 평균 값 차이가 더 크다. </font>

수치형 변수 모두 종속변수를 예측하는 데 사용할 만하다고 보인다.

<br>

<br>

다음은 각 범주형 변수별로 범주값별 종속변수 평균을 확인해보자.

범주값에 따라 종속 변수가 1인 비율이 차이가 있는지 간단히 확인하는 방법이다.

<br>

In [33]:
for col in columns_cat:
    print(train_df.groupby(col)[columns_y].mean().sort_values(by=columns_y, ascending=False), end='\n\n')

                    target
workclass                 
Self-emp-inc      0.558659
Federal-gov       0.387063
Local-gov         0.294630
Self-emp-not-inc  0.285714
State-gov         0.268960
Private           0.218792
Without-pay       0.000000

                         target
marital_status                 
Married-AF-spouse      0.476190
Married-civ-spouse     0.454959
Divorced               0.107262
Widowed                0.096735
Married-spouse-absent  0.083784
Separated              0.070288
Never-married          0.048324

                     target
occupation                 
Exec-managerial    0.485220
Prof-specialty     0.448489
Protective-serv    0.326087
Tech-support       0.304825
Sales              0.270647
Craft-repair       0.225310
Transport-moving   0.202926
Adm-clerical       0.133835
Machine-op-inspct  0.124619
Farming-fishing    0.116279
Armed-Forces       0.111111
Handlers-cleaners  0.061481
Other-service      0.041096
Priv-house-serv    0.006993

                  

<br>

학습 데이터에서 종속변수가 1인 비율은 약 0.249이다.

위 결과를 보면 범주값별로 종속변수 평균이 다양하게 나타나서 해당 변수는 종속변수 예측이 사용할 만하다고 볼 수 있다.

<br>

참고로 native country 변수의 경우 41개의 고유한 값을 가진다.

이렇게 한 변수가 가질 수 있는 고유한 값의 개수를 카디널리티(cardinality) 라고 부르는데, <font color='orange'> 카디널리티가 높은 변수는 모델 성능에 악영향을 준다. </font>

보통 100개 이상의 고유한 값을 가지면 카디널리티가 높다고 하는데, 이 경우 해당 변수를 제거하거나 일부 범주를 합쳐 카디널리티를 낮추 수 있다.

또는 다루기 까다롭지만 encoding을 적용할 수도 있다.

<br>

여기서는 학습 데이터 양이 적지 않고, 변수 값들을 봤을 때 카디널리티에 의한 모델 성능 저하를 크게 우려하지 않아도 될 것으로 보인다.

범주형 변수들에 대해 label encoding을 진행하도록하자.

label encoding은 문자로 된 범주를 정수 형태로 바꾸어준다.

<br>

In [34]:
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train, X_test])

for col in columns_cat:
    label_encoder = LabelEncoder()
    label_encoder.fit(X[col])
    
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

<br>

label encoding을 한 데이터는 선형 모델에는 적합하지 않다.

label encoding한 변수 값이 의미를 가지는 것이 아니기 때문이다.

따라서 이어지는 데이터 모형 구축 시에는 비선형 모델인 RandomForest나 XGBoost 알고리즘을 사용하도록 하자.

<br>

<br>

### **데이터 모형 구축**

분류 예측 모형을 만들고자 데이터 분할을 진행한다.

주어진 학습데이터를 sklearn의 `train_test_split` 함수를 사용하여 학습 데이터와 검증 데이터로 다시 분류할 수 있다.

In [36]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

<br>

분할된 데이터로 수치형 변수의 Scaling을 진행한다.

<br>

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[columns_num])

X_tr[columns_num] = scaler.transform(X_tr[columns_num])
X_val[columns_num] = scaler.transform(X_val[columns_num])
X_test[columns_num] = scaler.transform(X_test[columns_num])

<br>

split이 되었으니 해당 데이터를 사용하여 학습을 진행한다.

sklearn 패키지를 사용하여 RandomForest, XGBoost 모델을 만들어보자.

default 설정으로 간단하게 모델을 만든 후 성능을 확인해보자.

<br>

In [42]:
from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier()
model_RF.fit(X_tr, y_tr.values.squeeze())

RandomForestClassifier()

In [46]:
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.squeeze())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

<br>

참고로 XGBoost 학습 시 `eval_set`을 사용하여 충분히 학습을 시키면서 과적합을 방지하도록 할 수 있다.

<br>

In [49]:
model_xgb2 = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
model_xgb2.fit(
    X_tr, 
    y_tr.values.squeeze(), 
    early_stopping_rounds=50, 
    eval_metric='auc', 
    eval_set=[(X_val, y_val)], 
    verbose=10
)

[0]	validation_0-auc:0.90022


/Users/wooy0ng/miniconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/wooy0ng/miniconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[10]	validation_0-auc:0.91115
[20]	validation_0-auc:0.91612
[30]	validation_0-auc:0.91877
[40]	validation_0-auc:0.91949
[50]	validation_0-auc:0.92101
[60]	validation_0-auc:0.92200
[70]	validation_0-auc:0.92236
[80]	validation_0-auc:0.92263
[90]	validation_0-auc:0.92295
[100]	validation_0-auc:0.92322
[110]	validation_0-auc:0.92333
[120]	validation_0-auc:0.92302
[130]	validation_0-auc:0.92290
[140]	validation_0-auc:0.92263
[150]	validation_0-auc:0.92232
[157]	validation_0-auc:0.92219


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

<br>

XGBoost에서 default와 위 코드와의 차이점을 확인해보자.

default에서 `n_estimators` 값은 100이다.

반면 위 코드에서 생성한 모델은 `n_estimators` 값을 1000으로 아주 크게 잡았다.

**이 경우 학습을 잘 할수는 있지만, 그만큼 과적합의 위험이 크다.**

<br>

따라서 Fitting할 때 `eval_set`과 `early_stopping_rounds` 값을 지정해주었다.

모델이 매 iteration 마다 `eval_set`을 이용하여 성능을 계산하다가

`early_stopping_rounds` 이상의 iteration 동안 성능 계산이 없으면 학습을 멈추게 된다.

<br>

<br>

초도 모델의 성능을 확인하기 위해 검증 데이터를 이용하여 ROC-AUC 값을 구해보자.

<br>

In [50]:
from sklearn.metrics import roc_auc_score

y_pred_proba_RF = model_RF.predict_proba(X_val)
y_pred_proba_xgb1 = model_xgb1.predict_proba(X_val)

score_RF = roc_auc_score(y_val, y_pred_proba_RF[:, 1])
score_xgb1 = roc_auc_score(y_val, y_pred_proba_xgb1[:, 1])

print(score_RF)
print(score_xgb1)

0.8941557776377678
0.923355363130375


<br>

랜덤포레스트 모델 결과는 0.891, XGBoost 모델 결과는 0.918로 XGBoost 모델의 성능이 약간 더 좋게 나왔다.

이 두 모델의 하이퍼 파라미터 튜닝을 통해 성능을 더 올려볼 것이다.

그 전에 새로 만든 capital_gain_yn, capital_loss_yn이 모델 예측에 도움을 주었는지 확인해보고자 한다.

<br>

In [51]:
pd.DataFrame({
    'feature': X_tr.columns,
    'rf': model_RF.feature_importances_,
    'xgb': model_xgb1.feature_importances_
})

,feature,rf,xgb
0,age,0.215252,0.029841
1,workclass,0.049436,0.023887
2,education_num,0.136600,0.107569
3,marital_status,0.073257,0.114756
4,occupation,0.086202,0.034235
5,relationship,0.120824,0.371144
6,race,0.016760,0.017999
7,sex,0.010192,0.030995
8,capital_gain,0.097711,0.151205
9,capital_loss,0.030034,0.074128


<br>

각 모델의 feature_importances_ 값이 변수 중요도를 나타낸다.

capital_gain, capital_loss, capital_gain_yn, capital_loss_yn 4개 변수의 중요도를 보면

모두 capital_gain, capital_loss가 더 중요하게 쓰임을 알 수 있다.

<br>

하이퍼 파라미터 튜닝 전에 capital_gain_yn, capital_loss_yn 변수를 제외하도록 하자

<br>

In [52]:
columns_remove = ['capital_gain_yn', 'capital_loss_yn']

X_tr = X_tr.drop(columns_remove, axis=1)
X_val = X_val.drop(columns_remove, axis=1)
X_test = X_test.drop(columns_remove, axis=1)

<br>

sklearn 패키지 model_selection 모듈의 `GridSearchSV` 함수를 사용해서 몇 개 하이퍼파라미터 조합을 시도해볼 수 있다.

`GridSearchCV` 함수 인자 중 

`estimator`는 튜닝에 사용할 기본 모델을, 

`params_grid`는 하이퍼 파라미터 조합을,

`cv`는 교차 검증 시 몇개의 fold를 사용할 지를 의미한다.

<br>

In [57]:
from sklearn.model_selection import GridSearchCV

params_grid = {
    'max_depth': [3, 5, 10],
    'min_child_weight': [1, 2],
}

xgb_cv = GridSearchCV(estimator=model_xgb1, param_grid=params_grid, cv=5)
xgb_cv.fit(X_tr, y_tr.values.squeeze())

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [3, 5, 10], 'min_child_weight': [1, 2]})

<br>

`best_params_` 속성을 불러와 가장 좋은 성능을 보인 하이퍼파라미터 조합으로 새로 모델을 생성하도록 하자.

<br>

In [58]:
xgb_cv.best_params_

{'max_depth': 5, 'min_child_weight': 1}

In [60]:
model_xgb = XGBClassifier(
    max_depth=5,
    min_child_weight=1
)

model_xgb.fit(X_tr, y_tr.values.squeeze(), eval_set=[(X_val, y_val)], eval_metric='auc', early_stopping_rounds=50, verbose=10)

[0]	validation_0-auc:0.87244
[10]	validation_0-auc:0.91279
[20]	validation_0-auc:0.92053


/Users/wooy0ng/miniconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/wooy0ng/miniconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[30]	validation_0-auc:0.92410
[40]	validation_0-auc:0.92461
[50]	validation_0-auc:0.92495
[60]	validation_0-auc:0.92546
[70]	validation_0-auc:0.92595
[80]	validation_0-auc:0.92547
[90]	validation_0-auc:0.92523
[99]	validation_0-auc:0.92517


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [61]:
model_xgb.best_score

0.9259779481648862

<br>

가장 좋은 성능을 보이는 모델의 predict_proba를 제출하도록 하자.

5만 달러 초과인 경우가 1이기 때문에 2차원의 값 중 두 번째 줄에 해당하는 것이 우리가 추출해야하는 확률 값이다.

<br>

In [62]:
pred_proba = model_xgb.predict_proba(X_test)[:, 1]
pd.DataFrame({
    'index': X_test.index,
    'target': pred_proba
}).to_csv('../yemoonsaBigdata/res/003000000.csv', index=False)